<a href="https://colab.research.google.com/github/aliikhwan99/Different_stage_of_AD_detection/blob/main/Different_stage_of_AD_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'stage-of-alzheimer:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5850466%2F9592054%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241014%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241014T014818Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D81843272db38bc3ba676e8005e39cd49a38b0c8684382be1dfec57d53e295153fabcd2b787a9958c224a3d0beb4c4d82d8c8bb5350f728034114f286b5ac4e6ebb7298b809f739e67c945e84308e926e05893d2ddd246a966e11f740855f714c2c8c719510551dc7dcd6133092c9790318a7c2c7855c37626b64e0855d801c40b8e55f477806d849dcb5120ab13de12fedc14bb709866a2fee14ccb9bce1f866d5077a9cc55c33f5918af253a45133e088d2394822f78c8fa8cdac336baf1644fa70373f59eb961fa44b84dce0be43fb2932e59e07bb42747bee87beb48f7a743e97eeb6ee061f95e370530c3b905fc02aa607753dd84861ac4cc88878bf270c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# visulize rs-fMRI

In [ ]:
# Averaging the fMRI image across time
fmri_img_mean = image.mean_img(fmri_img)

# Project to surface
texture_left = surface.vol_to_surf(fmri_img_mean, fsaverage.pial_left)
texture_right = surface.vol_to_surf(fmri_img_mean, fsaverage.pial_right)

# Plot the projected data
plotting.plot_surf_stat_map(fsaverage.infl_left, texture_left, hemi='left', title='Left Hemisphere')
plotting.plot_surf_stat_map(fsaverage.infl_right, texture_right, hemi='right', title='Right Hemisphere')

plotting.show()


# ORGIGIN

In [ ]:
!apt-get install -y dcm2niix


In [ ]:
import subprocess

dicom_directory = '/kaggle/input/stage-of-alzheimer/NC/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623'
output_directory = '/kaggle/working'  # Specify where you want to save the NIfTI file

# Run dcm2niix and capture output and error messages
result = subprocess.run(['dcm2niix', '-z', 'y', '-f', '%p_%s', '-o', output_directory, dicom_directory],
                        capture_output=True, text=True)

# Print the output and error messages
print("Output:\n", result.stdout)
print("Error:\n", result.stderr)


In [ ]:
!apt-get install -y dcm2niix


In [ ]:
import os

print("Converted files:")
print(os.listdir(output_directory))


In [ ]:
from nilearn import image, plotting

# Load the NIfTI file
nifti_file = os.path.join(output_directory, 'Resting_State_fMRI_501a.nii.gz')
img = image.load_img(nifti_file)

# Extract the first time point (index 0)
first_time_point = img.slicer[..., 0]  # Select the first time point

# Visualize the first time point
plotting.plot_img(first_time_point, display_mode='z', cut_coords=[-20, -10, 0])
plotting.show()


In [ ]:
from nilearn import image, plotting

# Load the NIfTI file
nifti_file = os.path.join(output_directory, 'Resting_State_fMRI_501a.nii.gz')
img = image.load_img(nifti_file)

# Compute the mean across all time points
mean_img = image.mean_img(img)

# Visualize the mean image
plotting.plot_img(mean_img, display_mode='z', cut_coords=[-20, -10, 0])
plotting.show()


# Normal

# Visualizing the Connectivity Matrix

# varation 2

In [ ]:
import os
import numpy as np
import pydicom  # Make sure pydicom is installed in your environment
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import datasets
from scipy.ndimage import gaussian_filter  # Import Gaussian filter

# Path to the directory containing the DICOM files
dicom_directory = '/kaggle/input/stage-of-alzheimer/NC/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623'

# Check if the directory exists
if not os.path.exists(dicom_directory):
    print(f"Directory not found: {dicom_directory}")
else:
    print(f"Directory exists: {dicom_directory}")

    # Load the DICOM files
    def load_dicom_series(directory):
        dicom_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.dcm')]
        dicom_files.sort()  # Sort the files to maintain order
        images = []
        for dicom_file in dicom_files:
            ds = pydicom.dcmread(dicom_file)
            images.append(ds.pixel_array)  # Append pixel data
        return np.array(images)

    # Load your fMRI data
    fmri_data = load_dicom_series(dicom_directory)
    print(f"Shape of fMRI data: {fmri_data.shape}")  # (time_points, height, width)

# Function to extract time series from the 4D fMRI data
def extract_time_series(fmri_data):
    # Get the shape of the fMRI data
    time_points, height, width = fmri_data.shape[0], fmri_data.shape[1], fmri_data.shape[2]

    # Reshape to a 2D array: (voxels, time_points)
    reshaped_data = fmri_data.reshape(-1, time_points)
    return reshaped_data

# Extract time series from the fMRI data
time_series = extract_time_series(fmri_data)
print(f"Shape of time series data: {time_series.shape}")  # (voxels, time_points)

# Function to compute the functional connectivity matrix
def compute_functional_connectivity(time_series):
    # Calculate the correlation matrix
    connectivity_matrix = np.corrcoef(time_series)
    return connectivity_matrix

# Compute the functional connectivity matrix
connectivity_matrix = compute_functional_connectivity(time_series)
print(f"Shape of connectivity matrix: {connectivity_matrix.shape}")  # (voxels, voxels)

# Apply Gaussian filter to the functional connectivity matrix
smoothed_connectivity_matrix = gaussian_filter(connectivity_matrix, sigma=2)  # You can adjust sigma value

# Load the AAL brain atlas and its labels
atlas = datasets.fetch_atlas_aal()
atlas_labels = atlas['labels']  # Get brain region labels

# Truncate the connectivity matrix if it's larger than the number of available atlas labels
num_labels = len(atlas_labels)
if smoothed_connectivity_matrix.shape[0] > num_labels:
    truncated_connectivity_matrix = smoothed_connectivity_matrix[:num_labels, :num_labels]
else:
    truncated_connectivity_matrix = smoothed_connectivity_matrix  # Use the original if no truncation needed

# Function to plot a clearer connectivity matrix with enhancements
def plot_enhanced_connectivity_matrix(connectivity_matrix, labels):
    plt.figure(figsize=(14, 12), dpi=100)  # Increase size and DPI
    sns.set(font_scale=1)  # Increase font scale for better readability
    sns.heatmap(connectivity_matrix, cmap='coolwarm', square=True,
                xticklabels=labels[:connectivity_matrix.shape[0]],
                yticklabels=labels[:connectivity_matrix.shape[0]],
                cbar_kws={"shrink": .4}, linewidths=.2)  # Add lines between cells for clarity
    plt.title('Functional Connectivity Matrix (Enhanced)', fontsize=5)
    plt.xlabel('Brain Regions', fontsize=4)
    plt.ylabel('Brain Regions', fontsize=4)
    plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to fit everything well
    plt.show()

# Plot the enhanced connectivity matrix with the AAL labels
plot_enhanced_connectivity_matrix(truncated_connectivity_matrix, atlas_labels)


In [ ]:
import os
import numpy as np
import pydicom  # Make sure pydicom is installed in your environment
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import datasets
from scipy.ndimage import gaussian_filter  # Import Gaussian filter
from scipy.ndimage import median_filter

# Path to the directory containing the DICOM files
dicom_directory = '/kaggle/input/stage-of-alzheimer/NC/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623'

# Check if the directory exists
if not os.path.exists(dicom_directory):
    print(f"Directory not found: {dicom_directory}")
else:
    print(f"Directory exists: {dicom_directory}")

    # Load the DICOM files
    def load_dicom_series(directory):
        dicom_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.dcm')]
        dicom_files.sort()  # Sort the files to maintain order
        images = []
        for dicom_file in dicom_files:
            ds = pydicom.dcmread(dicom_file)
            images.append(ds.pixel_array)  # Append pixel data
        return np.array(images)

    # Load your fMRI data
    fmri_data = load_dicom_series(dicom_directory)
    print(f"Shape of fMRI data: {fmri_data.shape}")  # (time_points, height, width)

# Function to extract time series from the 4D fMRI data
def extract_time_series(fmri_data):
    # Get the shape of the fMRI data
    time_points, height, width = fmri_data.shape[0], fmri_data.shape[1], fmri_data.shape[2]

    # Reshape to a 2D array: (voxels, time_points)
    reshaped_data = fmri_data.reshape(-1, time_points)
    return reshaped_data

# Extract time series from the fMRI data
time_series = extract_time_series(fmri_data)
print(f"Shape of time series data: {time_series.shape}")  # (voxels, time_points)

# Function to compute the functional connectivity matrix
def compute_functional_connectivity(time_series):
    # Calculate the correlation matrix
    connectivity_matrix = np.corrcoef(time_series)
    return connectivity_matrix

# Apply median filter to reduce noise
filtered_matrix = median_filter(connectivity_matrix, size=3)  # Adjust size for the window size

# Compute the functional connectivity matrix
connectivity_matrix = compute_functional_connectivity(time_series)
print(f"Shape of connectivity matrix: {connectivity_matrix.shape}")  # (voxels, voxels)

# Apply Gaussian filter to the functional connectivity matrix
smoothed_connectivity_matrix = gaussian_filter(connectivity_matrix, sigma=3)  # You can adjust sigma value

# Load the AAL brain atlas and its labels
atlas = datasets.fetch_atlas_aal()
atlas_labels = atlas['labels']  # Get brain region labels

# Truncate the connectivity matrix if it's larger than the number of available atlas labels
num_labels = len(atlas_labels)
if smoothed_connectivity_matrix.shape[0] > num_labels:
    truncated_connectivity_matrix = smoothed_connectivity_matrix[:num_labels, :num_labels]
else:
    truncated_connectivity_matrix = smoothed_connectivity_matrix  # Use the original if no truncation needed

# Function to plot a clearer connectivity matrix with enhancements
def plot_enhanced_connectivity_matrix(connectivity_matrix, labels):
    plt.figure(figsize=(14, 12), dpi=100)  # Increase size and DPI
    sns.set(font_scale=1)  # Increase font scale for better readability
    sns.heatmap(connectivity_matrix, cmap='coolwarm', square=True,
                xticklabels=labels[:connectivity_matrix.shape[0]],
                yticklabels=labels[:connectivity_matrix.shape[0]],
                cbar_kws={"shrink": .4}, linewidths=.2)  # Add lines between cells for clarity
    plt.title('Functional Connectivity Matrix (Enhanced)', fontsize=5)
    plt.xlabel('Brain Regions', fontsize=4)
    plt.ylabel('Brain Regions', fontsize=4)
    plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to fit everything well
    plt.show()

# Plot the enhanced connectivity matrix with the AAL labels
plot_enhanced_connectivity_matrix(truncated_connectivity_matrix, atlas_labels)


# Actual brain activity position

In [ ]:
pip install nilearn matplotlib


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import plotting, datasets, surface
from nilearn.input_data import NiftiLabelsMasker

# Mock example of your connectivity matrix; replace with your actual matrix
connectivity_matrix = np.random.rand(90, 90)  # Example connectivity matrix (90 regions)

# Load the AAL atlas
atlas = datasets.fetch_atlas_aal()
atlas_filename = atlas['maps']  # AAL atlas NIfTI file
atlas_labels = atlas['labels']  # Get brain region labels

# Load a standard brain surface for visualization
fsaverage = datasets.fetch_surf_fsaverage()

# Use NiftiLabelsMasker to extract time series data based on atlas regions
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True)

# Prepare a 1D array to represent the functional connectivity on the brain surface
left_connectivity = np.zeros(45)  # For left hemisphere
right_connectivity = np.zeros(45)  # For right hemisphere

# Map connectivity values to left and right hemispheres
for i in range(len(atlas_labels)):
    if i < 45:  # Left hemisphere
        left_connectivity[i] = connectivity_matrix[i, :].mean()  # Example aggregation
    elif i < 90:  # Right hemisphere
        right_index = i - 45  # Adjust index for right hemisphere
        right_connectivity[right_index] = connectivity_matrix[i, :].mean()  # Example aggregation

# Project the AAL atlas regions onto the brain surface
texture_left = surface.vol_to_surf(atlas_filename, fsaverage.pial_left)
texture_right = surface.vol_to_surf(atlas_filename, fsaverage.pial_right)

# Plot the left hemisphere
plotting.plot_surf_stat_map(fsaverage['infl_left'],
                            stat_map=texture_left,
                            hemi='left',
                            view='lateral',
                            title='Left Hemisphere',
                            colorbar=True)

# Plot the right hemisphere
plotting.plot_surf_stat_map(fsaverage['infl_right'],
                            stat_map=texture_right,
                            hemi='right',
                            view='lateral',
                            title='Right Hemisphere',
                            colorbar=True)

plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot using the correct 3D axes
plotting.plot_surf_stat_map(
    fsaverage['infl_left'],
    stat_map=left_connectivity,
    view='lateral',
    threshold=0.1,
    colorbar=True,
    title='Functional Connectivity (Left Hemisphere)',
    axes=ax
)

plt.show()


# LMCI